# 1. Audiobooks

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/peerkart

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/peerkart


In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
from sklearn.preprocessing import StandardScaler

# 2. Import file
* read the csv
* make the ratio between 0 and 1 same
* repeat y=1 five times

In [4]:
df = pd.read_csv('Audiobooks_data.csv', header=None)
print(df.shape)
display(df[[11]].value_counts())

df0 = df[df[11]==0]
df1 = df[df[11]==1]
df1 = pd.concat([df1,df1,df1,df1,df1])

df = pd.concat([df1,df0]).reset_index(drop=True)
print('\n',df.shape)
display(df[[11]].value_counts())

(14084, 12)


11
0     11847
1      2237
dtype: int64


 (23032, 12)


11
0     11847
1     11185
dtype: int64

# 3. Split train, test and validation
* train = 70%
* test = 10%
* validation = 20%

In [5]:
df = df.reset_index()
df['index'] = df['index']%10
df['flag'] = np.where(df['index'].isin([9]), 'Test',
                      np.where(df['index'].isin([0,5]), 'Validate', 'Training'))
display(df['flag'].value_counts())

train = df[df['flag']=='Training']
test = df[df['flag']=='Test']
val = df[df['flag']=='Validate']

print('\n')
print(train.shape[0], np.sum(train[11]), np.round(np.sum(train[11]) / train.shape[0] * 100, 2))
print(test.shape[0], np.sum(test[11]), np.round(np.sum(test[11]) / test.shape[0] * 100, 2))
print(val.shape[0], np.sum(val[11]), np.round(np.sum(val[11]) / val.shape[0] * 100, 2))

Training    16122
Validate     4607
Test         2303
Name: flag, dtype: int64



16122 7830 48.57
2303 1118 48.55
4607 2237 48.56


# 4. Data is Scaled
* standard scalar is used
* standard scalar is trained on training
* applied on training, test and validation

In [6]:
trainx = train.drop(['index',0,11,'flag'], axis=1)
trainy = train[[11]]
scaler = StandardScaler()
scaler.fit(trainx)
trainx = pd.DataFrame(scaler.transform(trainx))
trainx['flag'] = 'Training'

testx = test.drop(['index',0,11,'flag'], axis=1)
testy = test[[11]]
testx = pd.DataFrame(scaler.transform(testx))
testx['flag'] = 'Test'

valx = val.drop(['index',0,11,'flag'], axis=1)
valy = val[[11]]
valx = pd.DataFrame(scaler.transform(valx))
valx['flag'] = 'Validate'

y = pd.concat([trainy, testy, valy]).reset_index(drop=True)
x = pd.concat([trainx, testx, valx]).reset_index(drop=True)
df = pd.concat([x,y], axis=1)
display(x.describe().transpose())

,count,mean,std,min,25%,50%,75%,max
0,23032.0,-0.005124,1.001958,-2.715024,-0.758204,0.111494,1.198616,1.198616
1,23032.0,-0.001333,1.001991,-1.869277,-0.743699,-0.243442,0.381880,6.009771
2,23032.0,0.000509,1.014955,-0.650070,-0.361271,-0.231606,0.163282,24.316296
3,23032.0,0.000775,1.003566,-0.692451,-0.473623,-0.268192,0.014647,18.225029
4,23032.0,-0.000947,0.999166,-0.444515,-0.444515,-0.444515,-0.444515,2.249645
5,23032.0,0.001322,0.990352,-11.473184,-0.009000,-0.009000,-0.009000,1.570768
6,23032.0,0.004063,1.004274,-0.385065,-0.385065,-0.385065,-0.334423,4.679063
7,23032.0,-0.000047,1.000112,-0.437885,-0.437885,-0.437885,-0.193407,7.548395
8,23032.0,0.000149,1.067278,-0.187311,-0.187311,-0.187311,-0.187311,67.521758
9,23032.0,0.003715,1.002320,-0.754531,-0.754531,-0.548691,0.621347,4.272299


# 5. Save the datasets

In [7]:
df.to_csv('01_preprocessing.csv', index=False)
df.shape

(23032, 12)